In [1]:
import openmc
import openmc.deplete as od
import math
import pandas as pd
import numpy as np

In [2]:
fuel = openmc.Material(name="uo2")
fuel.add_element("U", 1, percent_type="ao", enrichment=4.25)
fuel.add_element("O", 2)
fuel.set_density("g/cc", 10.4)

clad = openmc.Material(name="clad")
clad.add_element("Zr", 1)
clad.set_density("g/cc", 6)

water = openmc.Material(name="water")
water.add_element("O", 1)
water.add_element("H", 2)
water.set_density("g/cc", 1.0)
water.add_s_alpha_beta("c_H_in_H2O")
materials = openmc.Materials([fuel])

In [3]:
radii = [0.42, 0.45]d
fuel.volume = math.pi * radii[0] ** 2

In [4]:
cyl = openmc.ZCylinder(r=radii[0])
cyl_cell = openmc.Cell(region=-cyl, fill=fuel)
bound_box = openmc.rectangular_prism(1.24, 1.24, boundary_type="reflective")
root_cell = openmc.Cell(fill=openmc.Universe(cells=[cyl_cell]), region=bound_box)
geometry = openmc.Geometry([root_cell])

In [5]:
settings = openmc.Settings()
settings.particles = 1000
settings.inactive = 10
settings.batches = 50

In [6]:
model = openmc.model.Model(geometry, materials, settings)
operator = openmc.deplete.Operator(model, "../openmc/tests/chain_simple.xml")

In [13]:
nuclides = {}
for nuc,t in fuel.get_nuclide_atom_densities().items():
    nuclides[nuc] = t[1] * 1e24

In [9]:
micro_xs = od.FluxDepletionOperator.create_micro_xs_from_csv('./micro_xs_simple.csv')

In [10]:
test_op = od.FluxDepletionOperator(fuel.volume, nuclides, micro_xs, 1164719970082145.0, '../openmc/tests/chain_simple.xml',  dilute_initial=0.0)

In [11]:
power = 174
time_steps = [30]
integrator = openmc.deplete.PredictorIntegrator(test_op, time_steps, power, timestep_units='d')

In [12]:
#import pdb
#pdb.set_trace()
integrator.integrate()

# b /home/ooblack/projects/openmc/openmc/deplete/abc.py:795
# b /home/ooblack/projects/openmc/openmc/deplete/stepresults.py:498
# p conc
# p conc list
# c
# p results.data
# c
# p results.data

[openmc.deplete] t=0.0 s, dt=2592000 s, source=174
[openmc.deplete] t=2592000.0 (final operator evaluation)


In [55]:
results = openmc.deplete.Results.from_hdf5("./depletion_results.h5")

In [88]:
results.get_atoms('0', 'Cs135')

(array([      0., 2592000.]), array([1.46355381e+19, 1.55709141e+19]))

In [ ]:
pdb.set_trace()
results.export_to_materials(0, nuc_with_data=test_op._all_nuclides)

--Return--
None
> /tmp/ipykernel_27211/767852825.py(1)<module>()
----> 1 pdb.set_trace()
      2 results.export_to_materials(0, nuc_with_data=test_op._all_nuclides)

ipdb> b /home/ooblack/projects/openmc/openmc/deplete/results.py:398
Breakpoint 4 at /home/ooblack/projects/openmc/openmc/deplete/results.py:398
ipdb> c
    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

> /home/ooblack/projects/openmc/openmc/deplete/results.py(398)export_to_materials()
    396         # Overwrite material definitions, if they can be found in the depletion
    397         # results, and save them to the new depleted xml file.
4-> 398         for mat in mat_file:
    399             mat_id = str(mat.id)
    400             if mat_id in result.mat_to_ind:

ipdb> p mat
*** NameError: name 'mat' is not defined
ipdb> p mat_file
[Material
	ID             =	1
	Name           =	uo2
	Temperature    =	None
	Density        =	10.4 [g/c

In [90]:
results[0][0,'0','Cs135']

1.46355381175615e+19

In [45]:
fuel.get_nuclide_atom_densities()['U236'][1]*1e24

4.5724195495061115e+18

In [51]:
results.data

AttributeError: 'Results' object has no attribute 'data'

In [91]:
test_op._all_nuclides

['I135',
 'Xe135',
 'Xe136',
 'Cs135',
 'Gd157',
 'Gd156',
 'U234',
 'U235',
 'U238',
 'O16',
 'O17',
 'U236']